4 spheres

In [26]:
import numpy as np
import numpy.linalg as la
from scipy.sparse.linalg import eigsh

M = 1.
C = 1.
w0 = C / M
a = 1. # distance between spheres
x = np.array([[1.], [0.]]) # x hat
y = np.array([[0.], [1.]]) # y hat
a1 = np.sqrt(3) * a * x
a2 = (np.sqrt(3) * x + 3 * y) * a / 2.
R1 = 1 / 3 * (a1 + a2)
R2 = 1 / 3 * (-2 * a1 + a2)
R3 = 1 / 3 * (a1 - 2 * a2)
R1h = R1 / la.norm(R1)
R2h = R2 / la.norm(R2)
R3h = R3 / la.norm(R3)
R11 = R1h * R1h.conj().T
R22 = R2h * R2h.conj().T
R33 = R3h * R3h.conj().T
kxs = np.arange(-np.pi, np.pi, 0.1)


evals_all, evecs_all  = [], []
for kx in kxs:
    k = np.array([kx, 0.])
    p = np.exp(1.j * k.dot(a1))

    # BUild Hamiltonian
    H = np.zeros((8, 8), dtype=np.complex128)
    H[0:2, 2:4] = -R11 - R22 * p
    H[2:4, 4:6] = -R33
    H[4:6, 6:8] = -R11 - R22 * p
    H = H + H.conj().T
    for i in range(0, 4 * 2, 2):
        H[i:i+2, i:i+2] = R11 + R22 + R33

    # Solve eigenvalue equation
    evals, evecs = eigsh(H)
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all, evecs_all = np.array(evals_all), np.array(evecs_all)
evals_all.shape


# Plot
import plotly.graph_objects as go


data = []
for kx in kxs:
    for i in range(6):
        color = '#%02x%02x%02x' % (255 - int(i * 255 / 6), 0, int(i * 255 / 6))
        data.append(go.Scatter(x=kxs, y=evals_all[:, i], name=f"{i}", line=dict(color=color)))

fig = go.Figure(data=data)
fig.show()

